# Deep Learning: Image Classification Using Keras

### Task 1: Import Packages

In [ ]:
import os
import glob
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import keras_cv

### Task 2: Explore Directories Structure

In [ ]:
IMAGES_DIR = "data/images/"

dir_names = os.listdir(IMAGES_DIR)
img_paths_dict = dict()
for dir_name in dir_names:
    img_paths_dict[dir_name] = glob.glob(f"{IMAGES_DIR}{dir_name}/*")

for dir_name, file_paths in img_paths_dict.items():
    print(dir_name, len(file_paths))
    for file_path in file_paths[:5]:
        print(file_path)

### Task 3: Visualize Image Data

In [ ]:
for dir_name, file_paths in img_paths_dict.items():
    print(dir_name)
    for file_path in file_paths[:5]:
        image = plt.imread(file_path)
        plt.imshow(image)
        plt.show()

### Task 4: Create a Data Loader

In [ ]:
dataset = keras.utils.image_dataset_from_directory(
    IMAGES_DIR, image_size=(128, 128), label_mode="categorical"
)

### Task 5: Inspect Dataset Output

In [ ]:
images, labels = next(iter(dataset))

print(f"type: {type(images)}")
print(f"dtype: {images.dtype}")
print(f"shape: {images.shape}")

print(dataset.class_names[tf.math.argmax(labels[0])])
plt.imshow(images[0].numpy().astype("uint8"))
plt.show()

### Task 6: Create Train and Validation Datasets

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    IMAGES_DIR,
    image_size=(128, 128),
    validation_split=0.2,
    subset="training",
    label_mode="categorical",
    seed=0,
)
valid_ds = keras.utils.image_dataset_from_directory(
    IMAGES_DIR,
    image_size=(128, 128),
    validation_split=0.2,
    subset="validation",
    label_mode="categorical",
    seed=0,
)

### Task 7: Create the Model

In [ ]:
backbone = keras_cv.models.EfficientNetV2Backbone.from_preset("efficientnetv2_b0_imagenet")

model = keras_cv.models.ImageClassifier(
    backbone=backbone,
    num_classes=3,
    activation="softmax",
)

### Task 8: Define Loss and Metrics

In [ ]:
loss = keras.losses.CategoricalCrossentropy()
metric = keras.metrics.CategoricalAccuracy()

### Task 9: Compile the Model

In [ ]:
model.compile(loss=loss, metrics=metric)

### Task 10: Train the Model

In [ ]:
model.fit(train_ds, validation_data=valid_ds, epochs=2)

### Task 11: Evaluate the Model

In [ ]:
loss, acc = model.evaluate(valid_ds, verbose=False)
print(f"{loss=:.3}, {acc=:.3}")

### Task 12: Use Model on New Images

In [ ]:
image_paths = glob.glob(f"{IMAGES_DIR}/*/*")
image = plt.imread(random.choice(image_paths))

predictions = model.predict(image[None, ...], verbose=False)[0]
pred_cls = valid_ds.class_names[predictions.argmax()]

print(pred_cls)
plt.imshow(image)
plt.show()